In [2]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import numpy as np
import fiona
import rasterio.mask
import seaborn as sns

#######Utiliy Functions########

#Returns the file names of the three MODIS tiles for the given year and day-of-year
def getfilenames(doy, year):

    parts = [1,2,3]
    filestomos = []

    for part in parts:
        search = "{year}{part}/INT09GQ.A{year}{doy}.*.BIN.tif".format(part = part, doy = doy, year = year)
        filestomos.append(glob.glob(search)[0])

    return filestomos

#Given a year and doy, create a mosaic of the three MODIS tiles
def writemosaic(doy, year):
    src = getrepfilenames(doy)
    srclist = []
    for rast in src:
        rastfile = rasterio.open(rast)
        srclist.append(rastfile)


    mosaic, out_trans = merge(srclist)
    out_meta = srclist[0].meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 })

    outfp = 'C:/Users/wang_mc/Documents/SDG_Tests/gwp/reprojmos/A{year}{doy}.tif'.format(doy  = doy, year = year)
    with rasterio.open(outfp, 'w', **out_meta) as dest:
        dest.write(mosaic)



    return mosaic

#Clip selected mosaic for a particular doy and year and write to file
def clip2nam(doy, year):

    with fiona.open("C:/Users/wang_mc/Documents/SDG_Tests/namshp/NAM.shp", "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    src = rasterio.open('C:/Users/wang_mc/Documents/SDG_Tests/gwp/reprojmos/A{year}{doy}.tif'.format(year = year, doy = doy))
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    with rasterio.open("C:/Users/wang_mc/Documents/SDG_Tests/gwp/clipped/A{year}{doy}.tif".format(year = year, doy = doy), "w", **out_meta) as dest:
            dest.write(out_image)

#Returns a list of all the file names for a particular year
def getclippedfilenames(year):

    doylist = []
    for i in range(2, 366):
        doylist.append(str(i).zfill(3))

    filestomos = []

    for doy in doylist:
        search = "C:/Users/wang_mc/Documents/SDG_Tests/gwp/clipped/A{year}{doy}.*".format(year = year, doy = doy)
        filestomos.append(glob.glob(search)[0])

    return filestomos

#Returns a list of all the file names for a particular year
def getdailytotals(fileslist):
    water = []

    for filename in fileslist:
        with rasterio.open(filename) as src:
            water.append(src.read(1).sum()*250*250)

    return water

In [ ]:
#The following script creates a days indunated for the year 2018
filelist = getclippedfilenames(2018) # Get list of file names for 2018

sums = rasterio.open('gwp\clipped\A2013801.tif').read(1).astype('uint8') #Initialize a raster object with the first image

#For each image, add to the sum, thus for each pixel creating a days inundated out of the year
for f in filelist:
    sums += rasterio.open(f).read(1).astype('uint8') 

#Output
src = rasterio.open('gwp\clipped\A2018001.tif') #Open one file to get the metadata
meta = src.profile #Get metadata

with rasterio.open('gwp/output/2018sum.tif', 'w', **meta) as ff:
    ff.write(sums,1) #Write the days inundated raster with 

In [ ]:
#Draw a chart of the daily total water surface area for a given year
doy = range(2, 366)
filelist = getclippedfilenames(2018)

ax = sns.scatterplot(doy, getdailytotals(fileslist))
ax.set_title('Namibia Daily Total Water Surface Area')
ax.set(xlabel='Day of Year', ylabel='Area (Sqared Meter)')

In [14]:
#Calculates the total number of pixels in the summed image, i.e. Max extent of the year
src = rasterio.open('gwp/output/2018sum.tif').read(1) #Open Days inundated images
zero = np.zeros_like(src) #Create a zero array

maxextent = np.greater(src, zero) #Compare element wise, positive integers retrun true (value 1), otherwise false (value 0)
totalpixel = maxextent.sum() # Total number of pixels, multiply by pixel area for total area

In [17]:
#Calculate the difference between two years
src1 = rasterio.open('gwp/output/2013sum.tif').read(1)*10 #Multiply every element by 10 to the initial year. (Nowater = 0, Water = 10)
src2 = rasterio.open('gwp/output/2018sum.tif').read(1)

#Doing the addition operation between the two arrays has the following properties:
#Water in both 2013 and 2018 has value 11
#Water in neither 2013 nor 2018 has value 0
#Water only in 2013 has value 10
#Water only in 2018 has value 1
diff = np.add(src1, src2)

In [21]:
#This function compares and returns the total number of elements in an array with a particular value
#i.e. This will return number of pixels with water for particular years as compared to another when using the difference array
def arraycompsum(arr, val):
    valarr = np.zeros_like(arr)+val
    comparr = np.equal(arr, valarr)

    return comparr.sum()

In [28]:
#Example
#Number of water pixels in both 2013 and 2018
arraycompsum(diff, 11)

111